# Example for finetuning a transit lightcurve model based on measured data

This notebook creates a model based transit light curve, and tunes this light curve based on expiremental data from exoplanet HD209458b using the $\chi^{2}$ method.

## Credits

The methods used are described in Chapter 3 of _Transiting Exoplanets: Measuring the Properties of Planetary Systems_ by Haswell, taught at astronomy module [S382 Astrophysics](http://www.open.ac.uk/courses/modules/s382) of the the OpenUniversity, UK.

The data used is based on the observations of the transiting planet system c as presented in figure 2 of 
Charbonneau, D., Brown, T. M., Latham, D. W., & Mayor, M. 2000, ApJ, 529, L45.
It was found on [http://www.hao.ucar.edu/](http://www.hao.ucar.edu/research/stare/hd209458.html).

## Code

Use inline plots:

In [3]:
# %matplotlib inline

Load libraries:

In [4]:
import math
import numpy as np
import pandas as pd

import hkPhysicalConstants as hpc
import hkAstronomicalConstants as hac
import hkAstrophysics as hap

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

Load measurement data from file:

In [5]:
# Load measurement data
dataframe = pd.read_csv('./Data/hd209458b.csv', comment='#', header=0, names=['time', 'flux', 'error'], delim_whitespace=True)
dataframe['time'] = dataframe['time']*24*60*60 # Convert from fractions of a day to seconds

Basic data for H209458b:

In [6]:
# HD 209458 b
RS = 1.1 * hac.R_Sun # Radius host star
RP = 1.27 * hac.R_Jup # Radius orbiting planet
P = 3.52*24*60*60 # Period of the orbiting planet
a = 0.0467*hac.AU # Astrocentric semi-major axis
i = math.radians(87.1) # Inclination of the orbit with respect to observer
s = 'HD 209458 b' # Subject text

Define the limb darkening model to use:

In [7]:
# Use a logarithmic model
u = 0.5 # First parameter of limb-darkening law
v = -0.12 # Second parameter of limb darkening law
m = 'log' # Limb-darkening model to be used

Calculate a transit light curve, using basic data, at the measured data points:

In [8]:
# Use first estimates and calculate fit quality with measurement data
[timeArray, areaArray, fluxArray, relativeFluxArray] = hap.CreateLightCurve(RS = RS, RP = RP, P = P, a = a, i = i, u = u, v = v, model = m, timeArray = dataframe['time'])

Calculate $\chi^{2}$:

In [9]:
cur_chiSquared = hap.ChiSquared(relativeFluxArray, dataframe['flux'], dataframe['error'])

Modify the planet's **radius**, calculate a new light curve and check if $\chi^{2}$ decreases:

In [10]:
# Start modifying the radius of the exoplanet, and check if quality of fit with measurement data increases
prev_chiSquared = cur_chiSquared + 1
while cur_chiSquared<prev_chiSquared:
    prev_chiSquared = cur_chiSquared

    RP *= 1.01
    [timeArray, areaArray, fluxArray, relativeFluxArray] = hap.CreateLightCurve(RS = RS, RP = RP, P = P, a = a, i = i, u = u, v = v, model = m, timeArray = dataframe['time'])
    cur_chiSquared = hap.ChiSquared(relativeFluxArray, dataframe['flux'], dataframe['error'])
cur_chiSquared = prev_chiSquared

prev_chiSquared = cur_chiSquared + 1
while cur_chiSquared<prev_chiSquared:
    prev_chiSquared = cur_chiSquared

    RP /= 1.01
    [timeArray, areaArray, fluxArray, relativeFluxArray] = hap.CreateLightCurve(RS = RS, RP = RP, P = P, a = a, i = i, u = u, v = v, model = m, timeArray = dataframe['time'])
    cur_chiSquared = hap.ChiSquared(relativeFluxArray, dataframe['flux'], dataframe['error'])
cur_chiSquared = prev_chiSquared

Modify the orbit's **inclination**, calculate a new light curve and check if $\chi^{2}$ decreases:

In [11]:
# Start modifying the inclination of the exoplanet's orbit, and check if quality of fit with measurement data increases
prev_chiSquared = cur_chiSquared + 1
while cur_chiSquared<prev_chiSquared:
    prev_chiSquared = cur_chiSquared

    i *= 1.01
    [timeArray, areaArray, fluxArray, relativeFluxArray] = hap.CreateLightCurve(RS = RS, RP = RP, P = P, a = a, i = i, u = u, v = v, model = m, timeArray = dataframe['time'])
    cur_chiSquared = hap.ChiSquared(relativeFluxArray, dataframe['flux'], dataframe['error'])
cur_chiSquared = prev_chiSquared

prev_chiSquared = cur_chiSquared + 1
while cur_chiSquared<prev_chiSquared:
    prev_chiSquared = cur_chiSquared

    i /= 1.01
    [timeArray, areaArray, fluxArray, relativeFluxArray] = hap.CreateLightCurve(RS = RS, RP = RP, P = P, a = a, i = i, u = u, v = v, model = m, timeArray = dataframe['time'])
    cur_chiSquared = hap.ChiSquared(relativeFluxArray, dataframe['flux'], dataframe['error'])
cur_chiSquared = prev_chiSquared

Print the final parameters:

In [12]:
print ('Chi^2 = {0:.3G}; u = {1:.3G}; v = {2:.3G}; RP = {3:.3G} * R_Jup; i = {4:.3G}'.format(cur_chiSquared, u, v, RP/hac.R_Jup, math.degrees(i)))

Chi^2 = 121; u = 0.5; v = -0.12; RP = 1.39 * R_Jup; i = 87.1


Using these parameters, create a new **continuous** lightcurve (rather than only at the measured data points):

In [13]:
[timeArray, areaArray, fluxArray, relativeFluxArray] = hap.CreateLightCurve(RS = RS, RP = RP, P = P, a = a, i = i, u = u, v = v, model = m)

Plot the results:

In [14]:
# Plot values
x_min = min(timeArray)/(60*60*24)
x_max = max(timeArray)/(60*60*24)

fig = plt.figure()

# Plot a chart with the relative flux
min_y = []
min_y.append(min(relativeFluxArray))
min_y.append(min(dataframe['flux']) - max(dataframe['error']))
y_min = min(min_y)

max_y = []
max_y.append(max(relativeFluxArray))
max_y.append(max(dataframe['flux']) + max(dataframe['error']))
y_max = max(max_y)

y_min -= 0.1*(y_max-y_min)
y_max += 0.1*(y_max-y_min)

ax0 = fig.add_subplot(1, 1, 1)

ax0.plot(timeArray/(60*60*24), relativeFluxArray, color='blue', linewidth = 1)
ax0.errorbar(dataframe['time']/(60*60*24), dataframe['flux'], yerr=dataframe['error'], color='black', fmt='.')
# ax0.errorbar(dataframe['time']/(60*60*24), dataframe['flux'], yerr=dataframe['error'], color='black', fmt='-.')

ax0.set_xlim(x_min, x_max)
ax0.set_ylim(y_min, y_max)

ax0.set_xlabel('time/days')
ax0.set_ylabel('Relative flux')

plt.xlabel('time/days')
plt.show()